In [8]:
!pip install -U bitsandbytes==0.45.5 peft==0.10.0 trl==0.9.3 transformers datasets accelerate evaluate mlflow ipywidgets sentence-transformers



Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.3 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer, SFTConfig
import mlflow

# Sanity Check GPU explicitly
print("PyTorch Version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0))
print("Compute capability:", torch.cuda.get_device_capability(0))

# Explicit tensor calculation (quick GPU validation)
x = torch.rand(5000, 5000).cuda()
y = torch.rand(5000, 5000).cuda()
print("Explicit GPU tensor calculation successful:", (x @ y).mean().item())


PyTorch Version: 2.8.0.dev20250420+cu128
CUDA available: True
Device name: NVIDIA GeForce RTX 5090
Compute capability: (12, 0)
Explicit GPU tensor calculation successful: 1250.4169921875


In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:  17%|#6        | 828M/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

In [ ]:
!apt-get update && apt-get install -y iputils-ping
!ping -c 3 huggingface.co
